In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
%cd /content/drive/My Drive/Colab Notebooks/VehicleDetection
# %cd /content/drive/My Drive/Colab Notebooks/VehicleDetection/data-2019/test

/content/drive/My Drive/Colab Notebooks/VehicleDetection


In [12]:
import tensorflow as tf
from keras.models import load_model, Model

yolov3 = load_model('keras-yolo3-master/yolo_ROB535.h5')
for layer in yolov3.layers:
    layer.trainable = False

gmail_model = load_model('keras-yolo3-master/gmail_shorterlabels_10.h5')
for layer in gmail_model.layers:
    layer.trainable = False

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [13]:
from keras import backend as K
import cv2
import numpy as np

net_h, net_w = 416, 416

get_98th_layer_output = K.function([yolov3.layers[0].input],
                                    [yolov3.layers[98].output])

layer_name = 'conv_105'
intermediate_layer_model = Model(inputs=gmail_model.input,
                                 outputs=gmail_model.get_layer(layer_name).output)
# intermediate_output = intermediate_layer_model.predict(data)

NameError: ignored

In [0]:
# yolov3.trainable = False
# yolov3.summary()

In [0]:
def preprocess_input(image, net_h, net_w):
    new_h, new_w, _ = image.shape

    # determine the new size of the image
    if (float(net_w)/new_w) < (float(net_h)/new_h):
        new_h = (new_h * net_w)//new_w
        new_w = net_w
    else:
        new_w = (new_w * net_h)//new_h
        new_h = net_h

    # resize the image to the new size
    resized = cv2.resize(image[:,:,::-1]/255., (new_w, new_h))

    # embed the image into the standard letter box
    new_image = np.ones((net_h, net_w, 3)) * 0.5
    new_image[(net_h-new_h)//2:(net_h+new_h)//2, (net_w-new_w)//2:(net_w+new_w)//2, :] = resized
    new_image = np.expand_dims(new_image, 0)

    return new_image

In [0]:
photo_filename = 'data4YOLO/image_folder/00107_image.jpg'
# photo_filename = 'data4YOLO/image_folder/00020_image.jpg'
# photo_filename = 'data4YOLO/image_folder/00058_image.jpg'
# load and prepare image
image = cv2.imread(photo_filename)
image_h, image_w, _ = image.shape
process_image = preprocess_input(image, net_h, net_w)
layer_output = get_98th_layer_output([process_image])[0]
layer_output = intermediate_layer_model.predict([process_image])[0]

In [0]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from keras.models import Sequential
from tensorflow.keras import applications
from keras.layers import Dense, Dropout, Flatten

# Use this with pretrained Model
def feedforward_network():
    model = models.Sequential()
    model.add(layers.Flatten())
    model.add(layers.Dense(2024, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(3))
    model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_squared_error'])
    return model

# Another attempt at 3D Model

In [0]:
import numpy as np
import csv
import cv2
import sklearn
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras_preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from random import shuffle
from math import ceil
from IPython.core.debugger import set_trace
from sklearn import preprocessing
from numpy import expand_dims
import pdb

class Pipeline:
    def __init__(self, model=None, base_path=''):
        self.data = []
        self.model = model
        self.base_path = base_path
        self.epochs = 40
        self.batch_size = 128
        self.training_samples = []
        self.validation_samples = []
        self.csv_path = self.base_path + '/centroids.csv'
        self.model_path = 'models/'

    def getSamples(self):
        print('Running getSamples functions')
        with open(self.csv_path) as csvfile:
            reader = csv.reader(csvfile)
            next(reader)
            count = 0
            lines = []
            for line in reader:
                lines.append(line)
                count += 1
                if (count == 3):
                    count = 0
                    self.data.append(lines)
                    lines = []
        return None
    
    def split_data(self):
        train_samples, validation_samples = train_test_split(self.data, test_size=0.2)
        self.training_samples = train_samples
        self.validation_samples = validation_samples
        return None
    
    def generator(self, samples, batch_size=32, b_image_generate=1):
        # generator_iter = 30
        num_samples = len(samples)
        while 1: # Loop forever so the generator never terminates
            shuffle(samples)
            for offset in range(0, num_samples, batch_size):
                batch_samples = samples[offset:offset+batch_size]
                layer_outputs = []
                groundtruth_bounds = []
                for batch_sample in batch_samples: 
                    # groundtruth_bound = [0] * 3 
                    groundtruth_bound = []
                    for i in range(3):
                        obj = batch_sample[i]
                        groundtruth_bound.append(obj[1])
                  
                    groundtruth_bounds.append(groundtruth_bound)
                    # load the image and get output of yolov layer n
                    # pdb.set_trace()
                    split_name = batch_sample[0][0].split('/')
                    name = self.base_path + '/' + split_name[0] + '/' + split_name[1] + '_image.jpg'
                    image = cv2.imread(name)
                    if image is not None:
                        image_h, image_w, _ = image.shape
                        process_image = preprocess_input(image, net_h, net_w)
                        # layer_output = get_98th_layer_output([process_image])[0]
                        layer_output = intermediate_layer_model.predict([process_image])[0]
                        layer_outputs.append(layer_output)
                        intermediate_layer_model
                # FINAL 
                # pdb.set_trace()
                x_train = np.array(layer_outputs)
                y_train = np.array(groundtruth_bounds)
                yield (x_train, y_train)
                
    def train_generator(self, batch_size):
        return self.generator(samples = self.training_samples, batch_size = self.batch_size, b_image_generate=1)
    def validation_generator(self, batch_size):
        return self.generator(samples = self.validation_samples, batch_size = self.batch_size, b_image_generate=0)
    
    def run(self):
        self.split_data()
        checkpoint = ModelCheckpoint(self.model_path + self.model_name + 'best.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', period=1)
        callbacks_list = [checkpoint]
        train_len = len(self.training_samples)
        print(train_len)
        valid_len = len(self.validation_samples)
        print(valid_len)
        history = self.model.fit_generator(self.train_generator(self.batch_size),
            steps_per_epoch=ceil(train_len/self.batch_size),
            validation_data=self.validation_generator(self.batch_size),
            validation_steps=ceil(valid_len/self.batch_size),
            callbacks=callbacks_list,
            epochs=self.epochs, 
            verbose=1)
        self.model.save(self.model_path + self.model_name + '.h5')
        return (history)
    
    def plotError(self, history):
      # Get training and test loss histories
      training_loss = history.history['loss']
      test_loss = history.history['val_loss']
      # Create count of the number of epochs
      epoch_count = range(1, len(training_loss) + 1)
      # Visualize loss history
      plt.plot(epoch_count, training_loss, 'r--')
      plt.plot(epoch_count, test_loss, 'b-')
      plt.legend(['Training Loss', 'Test Loss'])
      plt.xlabel('Epoch')
      plt.ylabel('Loss')
      plt.show();  
      plt.savefig(self.model_path + self.model_name + '.jpg')
      plt.close()
      
def main():
    # -------------- Define the inputs to the Pipeline -------------- [S]
    model = feedforward_network()
    base_path = 'data/data-2019/trainval'
    # -------------- Define the inputs to the Pipeline -------------- [E]
    pipeline = Pipeline(model, base_path)
    pipeline.model_name = ('model' + str(0))
    pipeline.getSamples()
    history = pipeline.run()
    pipeline.plotError(history)
    
if __name__ == '__main__':
    main()


Running getSamples functions
3184
796
Epoch 1/40
ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'VarIsInitializedOp_48:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training_arrays.py", line 332, in predict_loop
    batch_outs = f(ins_batch)  File "/usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py", line 2959, in __call__
    if hasattr(get_session(), '_make_callable_from_options'):  File "/usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py", line 216, in get_session
    [tf.is_variable_initialized(v) for v in candidate_vars])  File "/usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py", line 216, in <listcomp>
    [tf.is_variable_initialized(v) for v in candidate_vars])  File "/usr/local/lib/p

# Failed Model T.T Using 15 Objects and 8 vertcies of x, y, z

In [0]:
import numpy as np
import csv
import cv2
import sklearn
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras_preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from random import shuffle
from math import ceil
from IPython.core.debugger import set_trace
from sklearn import preprocessing
from numpy import expand_dims
import pdb

class Pipeline:
    def __init__(self, model=None, base_path=''):
        self.data = []
        self.model = model
        self.base_path = base_path
        self.epochs = 20
        self.batch_size = 128
        self.training_samples = []
        self.validation_samples = []
        self.csv_path = self.base_path + '/3D_bound_boxes.csv'
        self.model_path = 'models/'

    def getSamples(self):
        print('Running getSamples functions')
        with open(self.csv_path) as csvfile:
            reader = csv.reader(csvfile)
            next(reader)
            count = 0
            lines = []
            for line in reader:
                lines.append(line)
                count += 1
                if (count == 15):
                    count = 0
                    self.data.append(lines)
                    lines = []
        return None
    
    def split_data(self):
        train_samples, validation_samples = train_test_split(self.data, test_size=0.2)
        self.training_samples = train_samples
        self.validation_samples = validation_samples
        return None
    
    def generator(self, samples, batch_size=32, b_image_generate=1):
        # generator_iter = 30
        num_samples = len(samples)
        while 1: # Loop forever so the generator never terminates
            shuffle(samples)
            for offset in range(0, num_samples, batch_size):
                batch_samples = samples[offset:offset+batch_size]
                layer_outputs = []
                groundtruth_bounds = []
                for batch_sample in batch_samples:  
                    # groundtruth_bound = [1] * 24 * 15 # (x,y,z) * 8 * 15
                    groundtruth_bound = []
                    for i in range(len(batch_sample)):
                        obj = batch_sample[i]
                        groundtruth_bound.append(obj[2])
                        groundtruth_bound.append(obj[3])
                        groundtruth_bound.append(obj[4])
                        groundtruth_bound.append(obj[5])
                        groundtruth_bound.append(obj[6])
                        groundtruth_bound.append(obj[7])
                        groundtruth_bound.append(obj[8])
                        groundtruth_bound.append(obj[9])
                        groundtruth_bound.append(obj[10])
                        groundtruth_bound.append(obj[11])
                        groundtruth_bound.append(obj[12])
                        groundtruth_bound.append(obj[13])
                        groundtruth_bound.append(obj[14])
                        groundtruth_bound.append(obj[15])
                        groundtruth_bound.append(obj[16])
                        groundtruth_bound.append(obj[17])
                        groundtruth_bound.append(obj[18])
                        groundtruth_bound.append(obj[19])
                        groundtruth_bound.append(obj[20])
                        groundtruth_bound.append(obj[21])
                        groundtruth_bound.append(obj[22])
                        groundtruth_bound.append(obj[23])
                        groundtruth_bound.append(obj[24])
                        groundtruth_bound.append(obj[25])
                        
                    groundtruth_bounds.append(groundtruth_bound)
                    # load the image and get output of yolov layer n
                    split_name = batch_sample[0][0].split('_')[0]
                    name = self.base_path + '/' + split_name + '_image.jpg'
                    image = cv2.imread(name)
                    if image is not None:
                        image_h, image_w, _ = image.shape
                        process_image = preprocess_input(image, net_h, net_w)
                        layer_output = get_98th_layer_output([process_image])[0]
                        # layer_output = yolov3([process_image])[0]
                        layer_outputs.append(layer_output)
                # FINAL 
                # pdb.set_trace()
                x_train = np.array(layer_outputs)
                y_train = np.array(groundtruth_bounds)
                yield (x_train, y_train)
                
    def train_generator(self, batch_size):
        return self.generator(samples = self.training_samples, batch_size = self.batch_size, b_image_generate=1)
    def validation_generator(self, batch_size):
        return self.generator(samples = self.validation_samples, batch_size = self.batch_size, b_image_generate=0)
    
    def run(self):
        self.split_data()
        checkpoint = ModelCheckpoint(self.model_path + self.model_name + 'best.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='max', period=1)
        callbacks_list = [checkpoint]
        train_len = len(self.training_samples)
        print(train_len)
        valid_len = len(self.validation_samples)
        print(valid_len)
        history = self.model.fit_generator(self.train_generator(self.batch_size),
            steps_per_epoch=ceil(train_len/self.batch_size),
            validation_data=self.validation_generator(self.batch_size),
            validation_steps=ceil(valid_len/self.batch_size),
            callbacks=callbacks_list,
            epochs=self.epochs, 
            verbose=1)
        self.model.save(self.model_path + self.model_name + '.h5')
        return (history)
    
    def plotError(self, history):
      # Get training and test loss histories
      training_loss = history.history['loss']
      test_loss = history.history['val_loss']
      # Create count of the number of epochs
      epoch_count = range(1, len(training_loss) + 1)
      # Visualize loss history
      plt.plot(epoch_count, training_loss, 'r--')
      plt.plot(epoch_count, test_loss, 'b-')
      plt.legend(['Training Loss', 'Test Loss'])
      plt.xlabel('Epoch')
      plt.ylabel('Loss')
      plt.show();  
      plt.savefig(self.model_path + self.model_name + '.jpg')
      plt.close()
      
def main():
    # -------------- Define the inputs to the Pipeline -------------- [S]
    model = feedforward_network()
    base_path = 'data/data-2019/trainval'
    # -------------- Define the inputs to the Pipeline -------------- [E]
    pipeline = Pipeline(model, base_path)
    pipeline.model_name = ('model' + str(0))
    pipeline.getSamples()
    history = pipeline.run()
    pipeline.plotError(history)
    
if __name__ == '__main__':
    main()


# Test the Code

In [48]:
%cd /content/drive/My Drive/Colab Notebooks/VehicleDetection
%cd /content/drive/My Drive/Colab Notebooks/VehicleDetection/keras-yolo3-master
from google.colab import files
files.upload()  #this will prompt you to upload the kaggle.json

/content/drive/My Drive/Colab Notebooks/VehicleDetection
/content/drive/My Drive/Colab Notebooks/VehicleDetection/keras-yolo3-master


Saving gmail_shorterlabels_10.h5 to gmail_shorterlabels_10 (1).h5


In [40]:
import tensorflow as tf
from keras.models import load_model, Model
import keras
# model = load_model('models/Bound3D_model.h5')
# model = tf.keras.models.load_model('models/Bound3D_model.h5')
model = tf.keras.models.load_model('models/model0best.h5')

In [0]:
from google.colab.patches import cv2_imshow
photo_filename = 'data4YOLO/image_folder/00107_image.jpg'
photo_filename = 'data4YOLO/image_folder/00020_image.jpg'
# photo_filename = 'data4YOLO/image_folder/00058_image.jpg'
# photo_filename = 'data4YOLO/image_folder/04458_image.jpg'
# load and prepare image
image = cv2.imread(photo_filename)
image_h, image_w, _ = image.shape

cv2_imshow(image)
process_image = preprocess_input(image, net_h, net_w)
layer_output = get_98th_layer_output([process_image])[0]
output = model.predict(layer_output)
print(output)

In [21]:
print(len(output[0]))

360


In [0]:
import numpy as np
from glob import glob
import csv
from random import randint
import math

predictions = []
count = 0
with open('template.csv') as csvfile:
    # set_trace()
    reader = csv.reader(csvfile)
    next(reader)
    for line in reader:
        if count % 50 == 0:
            print(count)
        else:
            print(count, end=" ")
        split_name = line[0].split('/')
        micro_path = split_name[0] + '/' + split_name[1]
        image = cv2.imread('data/data-2019/test/' + micro_path + '_image.jpg')
        image_h, image_w, _ = image.shape
        process_image = preprocess_input(image, net_h, net_w)
        # make prediction
        layer_output = get_98th_layer_output([process_image])[0]
        output = model.predict(layer_output)
        # pdb.set_trace()
        x = output[0][0]
        y = output[0][1]
        z = output[0][2]
        r = math.sqrt(x**2 + y**2 + z**2)
        theta = np.arctan(x/z)
        count += 1
        predictions.append([r, theta])
        next(reader)
        # if (count > 5):
        #   break
    # set_trace()
print(predictions)
################################ Generating CSV ################################
name = '3d_bounding_output.csv'
cnt = 0
with open(name, 'w') as f:
    writer = csv.writer(f, delimiter=',', lineterminator='\n')
    with open('template.csv') as csvfile:
        reader = csv.reader(csvfile)
        writer.writerow(['guid/image/axis', 'value'])
        next(reader)
        for line in reader:
            pdb.set_trace()
            output_type = line[0].split('/')[2]
            if output_type == 'r':
                writer.writerow([line[0], predictions[cnt][0]]) # r  
            else:
                writer.writerow([line[0], predictions[cnt][1] * np.pi * 2]) # theta
                cnt += 1

In [0]:
################################ Generating CSV ################################
name = '3d_bounding_output.csv'
cnt = 0
with open(name, 'w') as f:
    writer = csv.writer(f, delimiter=',', lineterminator='\n')
    with open('template.csv') as csvfile:
        reader = csv.reader(csvfile)
        writer.writerow(['guid/image/axis', 'value'])
        next(reader)
        for line in reader:
            # pdb.set_trace()
            output_type = line[0].split('/')[2]
            if output_type == 'r':
                writer.writerow([line[0], predictions[cnt][0]]) # r  
            else:
                writer.writerow([line[0], predictions[cnt][1] * np.pi * 2]) # theta
                cnt += 1